In [10]:
def dec2bin(num=0, length=1):
    binChar = bin(num)
    binChar = list(binChar[2:]) # 15 -> '0b1111'
    if len(binChar) < length:
        binChar = ['0' for ii in range(length - len(binChar))] + binChar
    binNum = [int(aa) for aa in binChar]
    return binNum
for i in range(16):
    print dec2bin(i, 9)


[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 1]
[0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 1]
[0, 0, 0, 0, 0, 0, 1, 1, 0]
[0, 0, 0, 0, 0, 0, 1, 1, 1]
[0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 1]
[0, 0, 0, 0, 0, 1, 0, 1, 0]
[0, 0, 0, 0, 0, 1, 0, 1, 1]
[0, 0, 0, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 1, 1, 0, 1]
[0, 0, 0, 0, 0, 1, 1, 1, 0]
[0, 0, 0, 0, 0, 1, 1, 1, 1]


In [11]:
A = range(1, 10)
B = range(11, 30)

newA = A[:5]+B[5:]
newB = B[:5]+A[5:]
print newA, newB

[1, 2, 3, 4, 5, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29] [11, 12, 13, 14, 15, 6, 7, 8, 9]


In [4]:
import re
a = ['b0_Bp1', 'b1_Bp2', 'sigma1_Bp1', 'sigma2_MGS_dev', 'b0Bp2', 'sigma2D_MGS_dev']
for aa in a:
    if re.match(r'b[0-9]*[-_AaBbPp0-9]*', aa):
        print aa
    if (re.match(r'sigma[\S]*', aa)) and (not re.match(r'sigma2D[\S]*', aa)):
        print 2, aa

b0_Bp1
b1_Bp2
2 sigma1_Bp1
2 sigma2_MGS_dev
b0Bp2


In [6]:
a, b = ('a', 'b')
print a, b

a b


In [12]:
import random
import math
import copy
import re
import pandas as pd
import os, os.path

def gen_term_pairs(selectedTerms, parmNames):
    # return a table of search param name and (sigmaVal, truncation level), e.g., 'Ap': sigmaAp, b0_ratio
    termpairs = {}
    noneSigmaTerm = {"A", "Mav", "Slope"}

    sigmaParms = [aa for aa in parmNames if re.match(r'sigma[\S]*', aa) and (not re.match(r'sigma2D[\S]*', aa))]
    truncRatioParms = [aa for aa in parmNames if re.match(r'b[0-9]*[-_AaBbPp0-9]*', aa)]

    fixedTermPairs = {"Ap": "b0_ratio", "Bp": "b0_ratio", "Am": "b0m_ratio", "Bn": "b0n_ratio"}

    for curTerm in selectedTerms:
        curSigma = ""
        curTruncRatio = ""
        if curTerm not in noneSigmaTerm:
            if curTerm in fixedTermPairs.keys():
                curSigma = r'sigma{}'.format(curTerm)
                curTruncRatio = fixedTermPairs[curTerm]
            else:
                for sigmaName in sigmaParms:
                    if curTerm in sigmaName:
                        curSigma = sigmaName
                        break
                for truncRatio in truncRatioParms:
                    if curTerm in truncRatio:
                        curTruncRatio = truncRatio
                        break
        termpairs[curTerm] = (curSigma, curTruncRatio)
    return termpairs

def covert_job_parms(termpairs, jobparms):
    '''
    termpairs: all the resist terms and its components, PRS is not supported yet
    jobparms: table of search param name and value
    return a table of search param name and (sigmaVal, truncation level), e.g., 'Ap': 30, 1.2
    '''
    jobinputs = {}
    for curTerm, val in termpairs.items():
        curSigmaName, curTruncRatioName = val
        curSigmaVal = 0.
        curTruncRatioVal = 0.
        if curSigmaName != "":
            assert(curSigmaName in jobparms.keys())
            curSigmaVal = jobparms[curSigmaName]
        if curTruncRatioName != "":
            assert(curTruncRatioName in jobparms.keys())
            curTruncRatioVal = jobparms[curTruncRatioName]
        jobinputs[curTerm] = (curSigmaVal, curTruncRatioVal)
    return jobinputs

if __name__ == "__main__":
    cwd = os.getcwd()
    infile = os.path.join(cwd, "GA_init_setting.xlsx")
    df = pd.read_excel(infile)

    parmMin = df[['parmName', 'parmMin']].set_index('parmName').to_dict().values()[0]
    parmMax = df[['parmName', 'parmMax']].set_index('parmName').to_dict().values()[0]

    useterms = ['AG1', 'AG2', 'Ap', 'Bp', 'Am', 'Bn', 'MG1', 'Slope']
    termpairs = gen_term_pairs(useterms, parmMin)
    for jobparms in [parmMin, parmMax]:
        print covert_job_parms(termpairs, jobparms)

{'Slope': (0.0, 0.0), 'Bn': (30.0, 0.10000000000000001), 'Am': (30.0, 1.7), 'Ap': (30.0, 0.69999999999999996), 'MG1': (30.0, 0.0), 'Bp': (30.0, 0.69999999999999996), 'AG2': (50.0, 0.0), 'AG1': (0.0, 0.0)}
{'Slope': (0.0, 0.0), 'Bn': (300.0, 0.5), 'Am': (300.0, 1.8999999999999999), 'Ap': (300.0, 1.5), 'MG1': (300.0, 0.0), 'Bp': (300.0, 1.5), 'AG2': (300.0, 0.0), 'AG1': (20.0, 0.0)}


In [31]:
# linspace samples
import numpy as np
a=np.linspace(0.2, 2.4, 32)
print a, len(a)
print np.linspace(0.2, 2.4, 5)
print np.linspace(0.2, 0.69677419, 8)
print np.linspace(1.90322581, 2.4, 8)
print "\n"

print np.linspace(0.2, 2.4, 5)
print np.linspace(0, 20, 32)
print np.linspace(50, 200, 64)

[ 0.2         0.27096774  0.34193548  0.41290323  0.48387097  0.55483871
  0.62580645  0.69677419  0.76774194  0.83870968  0.90967742  0.98064516
  1.0516129   1.12258065  1.19354839  1.26451613  1.33548387  1.40645161
  1.47741935  1.5483871   1.61935484  1.69032258  1.76129032  1.83225806
  1.90322581  1.97419355  2.04516129  2.11612903  2.18709677  2.25806452
  2.32903226  2.4       ] 32
[ 0.2   0.75  1.3   1.85  2.4 ]
[ 0.2         0.27096774  0.34193548  0.41290322  0.48387097  0.55483871
  0.62580645  0.69677419]
[ 1.90322581  1.97419355  2.04516129  2.11612903  2.18709678  2.25806452
  2.32903226  2.4       ]


[ 0.2   0.75  1.3   1.85  2.4 ]
[  0.           0.64516129   1.29032258   1.93548387   2.58064516
   3.22580645   3.87096774   4.51612903   5.16129032   5.80645161
   6.4516129    7.09677419   7.74193548   8.38709677   9.03225806
   9.67741935  10.32258065  10.96774194  11.61290323  12.25806452
  12.90322581  13.5483871   14.19354839  14.83870968  15.48387097
  16.1290322

In [32]:
import random
print random.randint(0, 99)

44


In [38]:
import numpy as np
a =  range(10)
acculumatedLen = lambda x,y: np.sum(x[0:y+1])
for ii, vv in enumerate(a):
    print acculumatedLen(a, ii)

0
1
3
6
10
15
21
28
36
45
